In [1]:
from model import get_classification_model
from word2vec import Word2VecBuilder
from dataloader import ASTEncoder, load
from torch_geometric.data import DataLoader, Data
import glob
from tqdm import tqdm
import torch.nn.functional as F
import torch

In [13]:
BASELINE_GIN_CLASSIFIER = {
    "type": "GraphClassifier",
    "name": "BASELINE_GIN",
    "encoder": {
        "type": "GraphComposite",
        "pooling": {
            "type": "sum"
        },
        "encoder": {
            "num_layers": 3,
            "hidden_channels": 128,
            "layer_type": "CGIN",
            "norm_type": "None",

        }
    },
    "classifier": {
        "layer_type": "MLP",
        "dropout": 0.5,
        "num_layers": 3
    }
}

In [14]:
BASELINE_GIN_CLASSIFIER["features"] = 150
BASELINE_GIN_CLASSIFIER["classes"] = 1
model = get_classification_model(BASELINE_GIN_CLASSIFIER).encoder.node_level_encoder

In [15]:
#model.load("checkpoint")

In [16]:
w2v = Word2VecBuilder({
    "vector_size": 100,
    "window": 3,
    "training_files": [],
    "cache_dir": "."
})
w2v.build_model()

Loading wordvectors.


In [17]:
ast = ASTEncoder({"cache_dir": "."})
ast.build_astdict()

In [9]:
import os
import pickle
import gzip
train_graphs = []
test_graphs = []


test_file_count = 0
for name in tqdm(glob.glob("cache/LINEVUL_TRAIN/*_1.cpg.pt.gz")):
    file = gzip.open(name,'rb')
    object_file = pickle.load(file)
    train_graphs.append(Data(x=torch.cat((object_file["astenc"], object_file["codeenc"]), dim=1), edge_index=object_file["edge_index"], y=object_file["y"]))
    test_file_count += 1

for name in tqdm(glob.glob("cache/LINEVUL_TRAIN/*_0.cpg.pt.gz")[:test_file_count]):
    file = gzip.open(name,'rb')
    object_file = pickle.load(file)
    train_graphs.append(Data(x=torch.cat((object_file["astenc"], object_file["codeenc"]), dim=1), edge_index=object_file["edge_index"], y=object_file["y"]))

test_file_count = 0
for name in tqdm(glob.glob("cache/LINEVUL_TEST/*_1.cpg.pt.gz")):
    file = gzip.open(name,'rb')
    object_file = pickle.load(file)
    test_graphs.append(Data(x=torch.cat((object_file["astenc"], object_file["codeenc"]), dim=1), edge_index=object_file["edge_index"], y=object_file["y"]))
    test_file_count += 1

for name in tqdm(glob.glob("cache/LINEVUL_TEST/*_0.cpg.pt.gz")[:test_file_count]):
    file = gzip.open(name,'rb')
    object_file = pickle.load(file)
    test_graphs.append(Data(x=torch.cat((object_file["astenc"], object_file["codeenc"]), dim=1), edge_index=object_file["edge_index"], y=object_file["y"]))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 593/593 [00:03<00:00, 174.38it/s]


In [11]:
import numpy as np



train_loader = DataLoader(train_graphs, 2, shuffle=True)
test_loader = DataLoader(test_graphs, 1, shuffle=False)


/Users/i534627/Projects/patchdetect/models/taintgraph/env/lib/python3.7/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [18]:
from torch.optim import Adam
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader
from torch_geometric.data import Batch

optimizer = Adam(model.parameters(), lr=0.0001)
def train(model, optimizer, loader, device):
    model.train()
    total_loss = 0
    total_loss_c = 0
    total_loss_o = 0
    total_loss_co = 0
    correct_o = 0
    
    for it, data in tqdm(enumerate(loader), total=len(loader)):
        optimizer.zero_grad()
        data = data.to(device)
        data.edge_index = data.edge_index.long()
        data.x = data.x.float()

        one_hot_target = data.y.view(-1)
        c_logs, o_logs, co_logs = model(data)
        uniform_target = torch.ones_like(c_logs, dtype=torch.float).to(device) / 2

        c_loss = F.kl_div(c_logs, uniform_target, reduction='batchmean')
        o_loss = F.nll_loss(o_logs, one_hot_target)
        co_loss = F.nll_loss(co_logs, one_hot_target)
        loss = 0.1 * c_loss + 0.8 * o_loss + 0.1 * co_loss

        pred_o = o_logs.max(1)[1]
        correct_o += pred_o.eq(data.y.view(-1)).sum().item()
        loss.backward()
        total_loss += loss.item() * 4
        total_loss_c += c_loss.item() * 4
        total_loss_o += o_loss.item() * 4
        total_loss_co += co_loss.item() * 4
        optimizer.step()

    
    num = len(loader.dataset)
    total_loss = total_loss / num
    total_loss_c = total_loss_c / num
    total_loss_o = total_loss_o / num
    total_loss_co = total_loss_co / num
    correct_o = correct_o / num
    return total_loss, total_loss_c, total_loss_o, total_loss_co, correct_o

def eval_acc(model, loader, device):
    model.eval()
    correct = 0
    correct_c = 0
    correct_o = 0
    for data in tqdm(loader):
        data = data.to(device)
        data.edge_index = data.edge_index.long()
        data.x = data.x.float()
        with torch.no_grad():
            c_logs, o_logs, co_logs = model(data)
            pred = co_logs.max(1)[1]
            pred_c = c_logs.max(1)[1] 
            pred_o = o_logs.max(1)[1] 
        correct += pred.eq(data.y.view(-1)).sum().item()
        correct_c += pred_c.eq(data.y.view(-1)).sum().item()
        correct_o += pred_o.eq(data.y.view(-1)).sum().item()

    acc_co = correct / len(loader.dataset)
    acc_c = correct_c / len(loader.dataset)
    acc_o = correct_o / len(loader.dataset)
    return acc_co, acc_c, acc_o


In [13]:
for epoch in range(1, 100):
    train_loss, loss_c, loss_o, loss_co, train_acc = train(model, optimizer, train_loader, "cpu")
    test_acc, test_acc_c, test_acc_o = eval_acc(model, test_loader, "cpu")
    print("train stuff", train_loss, loss_c, loss_o, loss_co, train_acc)
    print("test stuff", test_acc, test_acc_c, test_acc_o)
    torch.save(model.state_dict(), "{}_model.chkpt".format(epoch))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1186/1186 [00:11<00:00, 104.42it/s]


train stuff 1.246224444717365 0.019376033110740154 1.3813290061870296 1.3922361540828685 0.5754111842105263
test stuff 0.6172006745362564 0.3912310286677909 0.53035413153457


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1186/1186 [00:09<00:00, 120.92it/s]


train stuff 1.2324029550103373 0.0006426434424754821 1.3691163835949018 1.3704556262120604 0.5771381578947369
test stuff 0.5219224283305227 0.45868465430016864 0.5236087689713322


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1186/1186 [00:08<00:00, 135.93it/s]


...

train stuff 1.215125473118142 6.011319591810829e-06 1.3501021148931038 1.3504316163494399 0.5785361842105263
test stuff 0.6281618887015177 0.418212478920742 0.5337268128161888


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1186/1186 [00:09<00:00, 126.66it/s]


train stuff 1.2165244020619674 4.719241865371403e-06 1.3515932120872955 1.3524934302917437 0.5826480263157895
test stuff 0.6290050590219224 0.5674536256323778 0.6197301854974705


 13%|████████████████████████████████▌                                                                                                                                                                                                                          | 788/6080 [00:23<02:40, 32.95it/s]


KeyboardInterrupt: 

In [19]:
model.load_state_dict(torch.load("14_model.chkpt"))
model.eval()

CGINEncoder(
  (layer): CausalGIN(
    (bn_feat): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_feat): GCNConv(150, 128)
    (bns_conv): ModuleList()
    (convs): ModuleList(
      (0): GINConv(nn=Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Linear(in_features=128, out_features=128, bias=True)
        (4): ReLU()
      ))
      (1): GINConv(nn=Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Linear(in_features=128, out_features=128, bias=True)
        (4): ReLU()
      ))
      (2): GINConv(nn=Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=

In [8]:
def load_test(name):
    graphs = []
    test_file_count = 0
    for name in tqdm(glob.glob("{}/*_1.cpg.pt.gz".format(name))):
        file = gzip.open(name,'rb')
        object_file = pickle.load(file)
        graphs.append(Data(x=torch.cat((object_file["astenc"], object_file["codeenc"]), dim=1), edge_index=object_file["edge_index"], y=object_file["y"]))
        test_file_count += 1

    for name in tqdm(glob.glob("{}/*_0.cpg.pt.gz".format(name))[:test_file_count]):
        file = gzip.open(name,'rb')
        object_file = pickle.load(file)
        graphs.append(Data(x=torch.cat((object_file["astenc"], object_file["codeenc"]), dim=1), edge_index=object_file["edge_index"], y=object_file["y"]))
    return graphs

In [19]:
for name in glob.glob("cache/LINEVUL_*"):
    print(name)
    graphs = load_test(name)
    test_loader = DataLoader(graphs, 1, shuffle=False)
    test_acc, test_acc_c, test_acc_o = eval_acc(model, test_loader, "cpu")
    print("test stuff", test_acc, test_acc_c, test_acc_o)


cache/LINEVUL_GOOGLE


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 596/596 [00:02<00:00, 240.40it/s]
0it [00:00, ?it/s]
/Users/i534627/Projects/patchdetect/models/taintgraph/env/lib/python3.7/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 596/596 [00:03<00:00, 164.03it/s]


test stuff 0.6342281879194631 0.4865771812080537 0.5989932885906041
cache/LINEVUL_TRAIN


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6080/6080 [00:25<00:00, 235.70it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6080/6080 [00:31<00:00, 196.05it/s]


test stuff 0.5036184210526315 0.5710526315789474 0.4776315789473684
cache/LINEVUL_CHROMIUM


...

test stuff 0.6139767054908486 0.519134775374376 0.5640599001663894
cache/LINEVUL_LLVM


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 596/596 [00:02<00:00, 199.36it/s]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 596/596 [00:03<00:00, 174.72it/s]


test stuff 0.6073825503355704 0.5184563758389261 0.5637583892617449
cache/LINEVUL_VAL


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 557/557 [00:03<00:00, 180.75it/s]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 557/557 [00:03<00:00, 169.99it/s]

test stuff 0.6481149012567325 0.4793536804308797 0.6050269299820467


In [59]:
from torch_geometric.utils import to_dense_adj

model.eval()
# https://github.com/php/php-src/commit/2938329ce19cb8c4197dec146c3ec887c6f61d01
for name in tqdm(glob.glob("cache/LINEVUL_TEST/179598_1*")):
    print(name)
    object_file = pickle.load(gzip.open(name))
    data = Data(x=torch.cat((object_file["astenc"], object_file["codeenc"]), dim=1), edge_index=object_file["edge_index"], y=object_file["y"])
    data.edge_index = data.edge_index.long()
    data.x = data.x.float()
    c_logs, o_logs, co_logs = model(data)
    print(o_logs.exp()[0][1])
    print(o_logs.exp())
    print(graph)
    
    edge_c, edge_t, node_c, node_t = model.layer.explain(data)
    node_mask = node_c.sigmoid()
    edge_mask = to_dense_adj(data.edge_index, edge_attr=edge_c)[0]
    for i in torch.topk(node_mask, 20).indices:
        #print("flows from")
        #for j in torch.topk(edge_mask.T[i], 4).indices:
        #    print(explaing.file[j], explaing.lines[j])
        #print("over")
        print( object_file["ast"][i] + " " + object_file["code"][i] + object_file["lines"][i] + "\n")
        #print("flowing to")
        #for j in torch.topk(edge_mask[i], 4).indices:
        #    print(explaing.file[j], explaing.lines[j])
    
        

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 57.11it/s]

cache/LINEVUL_TEST/179598_1.cpg.pt.gz
tensor(0.6698, grad_fn=<SelectBackward0>)
tensor([[0.3302, 0.6698]], grad_fn=<ExpBackward0>)
cache/LINEVUL_TEST/182081_1.cpg.pt.gz
"ProblemExpression" "php_error_docref(NULL TSRMLS_CC, E_WARNING, You must have 3x3 array);""25:25 1:72"

"DeclaredReferenceExpression" "IS_ARRAY""24:24 23:31"

"DeclaredReferenceExpression" "IS_ARRAY""23:23 104:112"

"ProblemExpression" "php_error_docref(NULL TSRMLS_CC, E_WARNING, You must have 3x3 array);""18:18 1:72"

"UnaryOperator" "&var2""30:30 59:64"

"UnaryOperator" "&var""23:23 66:70"

"ParamVariableDeclaration" """ "

"ParamVariableDeclaration" """ "

"BinaryOperator" "Z_TYPE_PP(var) == IS_ARRAY""23:23 86:112"

"ParamVariableDeclaration" """ "

"ParamVariableDeclaration" """ "

"ParamVariableDeclaration" """ "

"DeclaredReferenceExpression" "SUCCESS""23:23 75:82"

"DeclaredReferenceExpression" "SUCCESS""30:30 69:76"

"ParamVariableDeclaration" """ "

"BinaryOperator" "Z_TYPE_PP(var) != IS_ARRAY""24:24 5:31"

"C

In [105]:
import csv
csv.field_size_limit(1000000000)
groundtruth = {}
with open("../test.csv", 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in tqdm(reader):
            if row["target"] == "0":
                continue
            processed = row["processed_func"]
            groundtruth[row["index"]] = row["flaw_line_index"].split(",")

18864it [00:14, 1332.57it/s]


In [43]:
!pip install python-Levenshtein



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [150]:
import os
import pickle
import gzip
from torch_geometric.utils import to_dense_adj
from Levenshtein import distance as levenshtein_distance

top1=0
top5=0
top10=0
ifa=[]
num = 0
for name in tqdm(glob.glob("cache/LINEVUL_TEST/*_1*")):
    #print(name)
    idx = name.split("/")[-1].split("_")[0]
    object_file = pickle.load(gzip.open(name))
    data = Data(x=torch.cat((object_file["astenc"], object_file["codeenc"]), dim=1), edge_index=object_file["edge_index"], y=object_file["y"])
    data.edge_index = data.edge_index.long()
    data.x = data.x.float()
    c_logs, o_logs, co_logs = model(data)

    edge_c, edge_t, node_c, node_t = model.layer.explain(data)
    node_mask = node_c.sigmoid()
    count = 0
    if groundtruth[idx][0] == "":
        continue
    for i in torch.topk(node_mask, len(node_mask)).indices:
        if ":" not in object_file["lines"][i]:
            continue
        count += 1
        lower = object_file["lines"][i].split(" ")[0].split(":")[0].replace("\"","")
        upper = object_file["lines"][i].split(" ")[0].split(":")[1].replace("\"","")
        r = list(range(int(lower), int(upper)))
        hit = 0
        for ground in groundtruth[idx]:
            if ground == "":
                continue
            ground = int(ground)
            if ground in r:
                hit = 1
                ifa.append(count)
                break
        if count <= 1 and hit:
            top1 += 1
        if count <= 5 and hit:
            top5 += 1
        if count <= 10 and hit:
            top10 += 1
        
        if hit:
            num += 1
            break
    
        

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 593/593 [00:10<00:00, 59.22it/s]


In [149]:
top10/num

0.16009280742459397

In [145]:
sum(ifa)/num

825.9582366589327